In [185]:
import numpy as np
from scipy.linalg import expm
from qutip import (Qobj, sigmaz, identity, destroy,
                   basis, zero_ket, qzero, tensor,
                   propagator)
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector

SZ = sigmaz()
SM = destroy(2)
SP = SM.dag()
I = identity(2)


def density_matrix(state_vector):
    return state_vector.proj()


def vectorized_rho(rho):
    vec_rho = np.array(rho.full()).flatten()
    norm = np.linalg.norm(vec_rho)
    dim = int(np.log2(rho.shape[0])) * 2
    dims = [[2]*dim, [1]*dim]
    return Qobj(vec_rho / norm, dims=dims)


def unvectorized_rho(vec_rho):
    if isinstance(vec_rho, Statevector):
        vec_rho = vec_rho.data
    elif isinstance(vec_rho, Qobj):
        vec_rho = vec_rho.full()
    dim = int(np.log2(np.sqrt(vec_rho.shape[0])))
    rho_matrix = np.reshape(vec_rho, (2**dim, 2**dim))
    return Qobj(rho_matrix, dims=[[2]*dim, [2]*dim])


def super_operator(H, lindblad_ops=None):
    I = identity(H.dims[0])
    super_op = tensor(H, I) - tensor(I, H.trans())

    if lindblad_ops:
        for L in lindblad_ops:
            L_dag = L.dag()
            super_op += tensor(L, L_dag) - 0.5 * (
                tensor(L_dag * L, I) + tensor(I, L_dag * L)
            )

    return super_op


def count_CNOTs(circuit):
    BASIS_GATES = ['cx', 'u1', 'u2', 'u3', 'id']
    circuit = transpile(circuit, basis_gates=BASIS_GATES)
    return circuit.count_ops()['cx']

In [186]:
class System:
    def __init__(self,
                 energies,
                 couplings,
                 ):
        self.energies = energies
        self.couplings = couplings

    @property
    def system_size(self):
        return len(self.energies)

    @property
    def state(self):
        return self._state

    @state.setter
    def state(self, population):
        dim = self.system_size
        population /= np.sum(population)
        self._state = zero_ket(2**dim)
        for i, pop in enumerate(population):
            basis_i = basis(2**dim, 2**i)
            self._state += np.sqrt(pop) * basis_i
        self._state.dims = [[2]*dim, [1]*dim]

    def H_sys(self):
        H = qzero(dimensions=[2]*self.system_size)
        kron_I = [I]*self.system_size
        for i, energy in enumerate(self.energies):
            operators = kron_I.copy()
            operators[i] = SZ
            kron_SZ = tensor(*operators)
            H += - 0.5 * energy * kron_SZ

            for j in range(i+1, self.system_size):
                operators_ij = kron_I.copy()
                operators_ij[i] = SP
                operators_ij[j] = SM
                kron_ij = tensor(*operators_ij)

                operators_ji = kron_I.copy()
                operators_ji[i] = SM
                operators_ji[j] = SP
                kron_ji = tensor(*operators_ji)

                H += self.couplings[i, j] * (kron_ij + kron_ji)
        return H

In [187]:
from scipy.linalg import toeplitz

epsilons = [1, 2]
J = toeplitz([0, 1])

sys = System(epsilons, J)

sys.H_sys()

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=True
Qobj data =
[[-1.5  0.   0.   0. ]
 [ 0.   0.5  1.   0. ]
 [ 0.   1.  -0.5  0. ]
 [ 0.   0.   0.   1.5]]

In [188]:
sys.state = [1, 0]

sys.state

Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]]

In [189]:
rho = density_matrix(sys.state)

rho

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

In [190]:
vectorized_rho(rho)

Quantum object: dims=[[2, 2, 2, 2], [1, 1, 1, 1]], shape=(16, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]

In [191]:
n_qubits = sys.system_size * 2
qc = QuantumCircuit(n_qubits)

vec_rho = vectorized_rho(rho)
initial_state = Statevector(vec_rho.full())
qc.initialize(initial_state, range(n_qubits))

qc.draw()

┌──────────────────────────────────────────────┐
q_0: ┤0                                             ├
     │                                              │
q_1: ┤1                                             ├
     │  Initialize(0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0) │
q_2: ┤2                                             ├
     │                                              │
q_3: ┤3                                             ├
     └──────────────────────────────────────────────┘

In [192]:
statevector = Statevector.from_instruction(qc)
result_rho = unvectorized_rho(statevector)

result_rho

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

In [193]:
super_op = super_operator(sys.H_sys())

super_op

Quantum object: dims=[[2, 2, 2, 2], [2, 2, 2, 2]], shape=(16, 16), type='oper', dtype=CSR, isherm=True
Qobj data =
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  2.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0. -1. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0. -2.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  3.  0.  0.  0.]
 [ 0.  0.  0. 

In [194]:
H_propagator = propagator(sys.H_sys(), 0.01)

H_propagator

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=False
Qobj data =
[[0.9998875+0.01499944j 0.       +0.j         0.       +0.j
  0.       +0.j        ]
 [0.       +0.j         0.9999375-0.0049999j  0.       -0.00999979j
  0.       +0.j        ]
 [0.       +0.j         0.       -0.00999979j 0.9999375+0.0049999j
  0.       +0.j        ]
 [0.       +0.j         0.       +0.j         0.       +0.j
  0.9998875-0.01499944j]]

In [195]:
result_sv = H_propagator * sys.state

result_sv

Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.       +0.j        ]
 [0.9999375-0.0049999j ]
 [0.       -0.00999979j]
 [0.       +0.j        ]]

In [196]:
result_sv_rho = result_sv.proj()

result_sv_rho

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j        ]
 [0.00000000e+00-0.j         9.99900004e-01+0.j
  4.99979165e-05+0.00999917j 0.00000000e+00-0.j        ]
 [0.00000000e+00-0.j         4.99979165e-05-0.00999917j
  9.99958329e-05+0.j         0.00000000e+00-0.j        ]
 [0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j        ]]

In [197]:
L_propagator = propagator(super_op, 0.01)

L_propagator

Quantum object: dims=[[2, 2, 2, 2], [2, 2, 2, 2]], shape=(16, 16), type='oper', dtype=Dense, isherm=False
Qobj data =
[[ 1.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j          9.99750015e-01+0.01999783j
  -1.49990521e-04+0.00999867j  0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j         -1.49990521e-04+0.00999867

In [198]:
result_rho = L_propagator * vec_rho
result_rho_unvec = unvectorized_rho(result_rho)

result_rho_unvec

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j        ]
 [0.00000000e+00+0.j         9.99900005e-01+0.j
  4.99977432e-05+0.00999917j 0.00000000e+00+0.j        ]
 [0.00000000e+00+0.j         4.99977432e-05-0.00999917j
  9.99954865e-05+0.j         0.00000000e+00+0.j        ]
 [0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j        ]]

**1. Basic: Just map unitary decomposition**

In [199]:
from scipy.linalg import polar
from qiskit.circuit.library import UnitaryGate

# unitary_part, _ = polar(L_propagator.full())
# U = UnitaryGate(unitary_part)

U = UnitaryGate(L_propagator.full())

qc = QuantumCircuit(n_qubits)
qc.initialize(initial_state, range(n_qubits))
qc.append(U, range(n_qubits))

print(f'CNOTs: {count_CNOTs(qc)}')
qc.decompose(reps=0).draw()

CNOTs: 110


┌──────────────────────────────────────────────┐┌──────────┐
q_0: ┤0                                             ├┤0         ├
     │                                              ││          │
q_1: ┤1                                             ├┤1         ├
     │  Initialize(0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0) ││  Unitary │
q_2: ┤2                                             ├┤2         ├
     │                                              ││          │
q_3: ┤3                                             ├┤3         ├
     └──────────────────────────────────────────────┘└──────────┘

In [200]:
evolved_sv = Statevector.from_instruction(qc)
evolved_sv_rho = unvectorized_rho(evolved_sv)

evolved_sv_rho

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j        ]
 [0.00000000e+00+0.j         9.99900005e-01+0.j
  4.99977432e-05+0.00999917j 0.00000000e+00+0.j        ]
 [0.00000000e+00+0.j         4.99977432e-05-0.00999917j
  9.99954865e-05+0.j         0.00000000e+00+0.j        ]
 [0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j        ]]

**2. Convert to Pauli String**

In [201]:
from itertools import product
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import SuzukiTrotter


def filtered_pauli_strings(labels, coefficients):
    pauli_items = zip(labels, coefficients)
    def count_I(x): return x[0].count('I')
    pauli_dict = {label: coeff
                  for label, coeff in sorted(pauli_items,
                                             key=count_I,
                                             reverse=True,
                                             )
                  if abs(coeff) > 1e-6
                  }
    pauli_labels = list(pauli_dict.keys())
    pauli_coeffs = list(pauli_dict.values())
    return pauli_labels, pauli_coeffs


pauli_ops = ['I', 'X', 'Y', 'Z']
PRODUCT = product(pauli_ops, repeat=sys.system_size*2)
pauli_labels = [''.join(p) for p in PRODUCT]
pauli_strings = [SparsePauliOp(label) for label in pauli_labels]

coefficients = []
for pauli in pauli_strings:
    norm = 2**(sys.system_size * 2)
    pauli_matrix = pauli.to_matrix()
    coeff = np.trace(pauli_matrix.conj().T @ super_op.full())
    coefficients.append(coeff / norm)

pauli_labels, pauli_coeffs = filtered_pauli_strings(pauli_labels, coefficients)

super_op_paulis = SparsePauliOp(pauli_labels, pauli_coeffs)
propagator_paulis = PauliEvolutionGate(super_op_paulis, 0.01)

print('Superoperator in Pauli basis:')
print(super_op_paulis)

qc = QuantumCircuit(n_qubits)
qc.initialize(initial_state, range(n_qubits))
qc.append(propagator_paulis, range(n_qubits))

print(f'CNOTs: {count_CNOTs(qc)}')
qc.decompose(reps=2).draw()

Superoperator in Pauli basis:
SparsePauliOp(['IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIXX', 'IIYY', 'XXII', 'YYII'],
              coeffs=[ 1. +0.j,  0.5+0.j, -1. +0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j,  0.5+0.j,
  0.5+0.j])
CNOTs: 18


┌──────────────────┐ ┌──────────┐┌─────────────┐┌─────────────┐
q_0: ─|0>─┤0                 ├─┤ Rz(0.02) ├┤0            ├┤0            ├
          │                  │ ├──────────┤│  Rxx(-0.01) ││  Ryy(-0.01) │
q_1: ─|0>─┤1                 ├─┤ Rz(0.01) ├┤1            ├┤1            ├
          │  disentangler_dg │┌┴──────────┤└┬────────────┤└┬────────────┤
q_2: ─|0>─┤2                 ├┤ Rz(-0.02) ├─┤0           ├─┤0           ├
          │                  │├───────────┤ │  Rxx(0.01) │ │  Ryy(0.01) │
q_3: ─|0>─┤3                 ├┤ Rz(-0.01) ├─┤1           ├─┤1           ├
          └──────────────────┘└───────────┘ └────────────┘ └────────────┘

In [202]:
evolved_sv = Statevector.from_instruction(qc)
evolved_sv_rho = unvectorized_rho(evolved_sv)

evolved_sv_rho

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j        ]
 [0.00000000e+00+0.j         9.99900003e-01+0.j
  0.00000000e+00+0.00999933j 0.00000000e+00+0.j        ]
 [0.00000000e+00+0.j         0.00000000e+00-0.00999933j
  9.99966667e-05+0.j         0.00000000e+00+0.j        ]
 [0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j        ]]